In [1]:
import datetime
import subprocess
import os

import pandas as pd
from graphreduce.node import GraphReduceNode, DynamicNode
from graphreduce.graph_reduce import GraphReduce
from graphreduce.enum import ComputeLayerEnum, PeriodUnit

/Users/wesmadrigal/projects/graphreduce/venv/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
files = [
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/cust.csv',
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/orders.csv',
]

for f in os.listdir(os.getcwd()):
    for _f in files:
        _fname = _f.split('/')[-1]
        if _fname == f or _fname in f:
            os.remove(f)

for f in files:
  subprocess.run(['wget', f])

--2024-08-30 13:00:47--  https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/cust.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35 [text/plain]
Saving to: ‘cust.csv’

     0K                                                       100% 3.34M=0s

2024-08-30 13:00:47 (3.34 MB/s) - ‘cust.csv’ saved [35/35]

--2024-08-30 13:00:47--  https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/orders.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191 [text/plain]
Savin

In [3]:
# Let's define some custom nodes
# and custom operations

In [27]:
class OrderNode(GraphReduceNode):
    def do_filters(self):
        self.df = self.df[self.df[self.colabbr('amount')] < 100000]

    def do_annotate(self):
        pass

    def do_normalize(self):
        pass

    def do_reduce(self, reduce_key):
        return self.prep_for_features().groupby(self.colabbr(reduce_key)).agg(
            **{
                self.colabbr('num_orders'): pd.NamedAgg(column=self.colabbr(self.pk), aggfunc='count'),
                self.colabbr('max_amount'): pd.NamedAgg(column=self.colabbr('amount'), aggfunc='max'),
                self.colabbr('min_amount'): pd.NamedAgg(column=self.colabbr('amount'), aggfunc='min')
            }
        )

    def do_labels(self, reduce_key):
        label_df = self.prep_for_labels().groupby(self.colabbr(reduce_key)).agg(
            **{
                # We can subsequently turn this into a boolean
                self.colabbr('label_orders'): pd.NamedAgg(column=self.colabbr(self.pk), aggfunc='count')
            }
        )
        label_df[self.colabbr('label_had_order')] = label_df[self.colabbr('label_orders')].apply(lambda x: 1 if x > 0 else 0)
        return label_df

    def do_post_join_annotate(self):
        pass

    def do_post_join_filters(self):
        pass

In [19]:
class CustomerNode(GraphReduceNode):
    def do_filters(self):
        pass
    def do_annotate(self):
        pass
    def do_normalize(self):
        pass
    def do_reduce(self, reduce_key):
        pass
    def do_labels(self, reduce_key):
        pass
    def do_post_join_annotate(self):
        pass
    def do_post_join_filters(self):
        pass

In [20]:
# Instantiate order node

In [28]:
order = OrderNode(
    fpath='orders.csv',
    prefix='ord',
    date_key='ts',
    pk='id',
    fmt='csv',
    compute_layer=ComputeLayerEnum.pandas,
    compute_period_val=180,
    compute_period_unit=PeriodUnit.day,
    cut_date=datetime.datetime(2023,10,1),
    label_period_val=30,
    label_period_unit=PeriodUnit.day
)

In [30]:
order.do_data()

In [31]:
order.do_reduce('customer_id')

,ord_num_orders,ord_max_amount,ord_min_amount
ord_customer_id,,,
1,3,1200,10
3,1,220,220


In [32]:
order.do_labels('customer_id')

,ord_label_orders,ord_label_had_order
ord_customer_id,,
2,1,1
